# Demonstration and Comparison of Outlier Detection Methods

This Jupyter notebook allows to compare different outlier detection methods in terms of the ROC-AUC score.

We train unsupervised models on _MS1M_ dataset.
We calculate features given by these models on the validation dataset and on the outlier dataset (`cplfw_anime_outliers`) and compare the distance between the resulting pair of distributions of features.

In [1]:
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import pandas as pd
import os
import sys

if os.getcwd().endswith('quadrics') is not True:
    os.chdir('../')

In [53]:
# parameters for consistecny with the paper's notation
datasets_rename = {'calfw': 'CALFW','flickr': 'FFQH', 'ms1m': 'MS1M-ArcFace', 'cplfw': 'CPLFW', 'megaface': 'MegaFace'}
columns_dict = {'OneClassSVM': 'OCSVM', 'PCA': 'PCA', 'norms': 'NORM', 'quadrics': 'Q-FULL', 'quadrics_alg': 'Q-SUB'}
col_order = ['initial', 'Q-FULL', 'PCA', 'Q-SUB', 'OCSVM', 'NORM']
dataset_order = {i: j for j, i in enumerate(['MS1M-ArcFace', 'MegaFace', 'FFQH', 'CPLFW','CALFW'])}

### Calculation of Features
Choose methods to train (`methods_train`), methods for comparison (`methods`) and the set of validation datasets (`datasets`).

**Note:** we discard the pictures that cannot be recognized by a human from _CPLFW_ dataset, when it is considered as part of the validation.

In [36]:
with open("config.json", "r") as read_file:
    config_test = json.load(read_file)['test_params']

methods_train = ['OneClassSVM', 'PCA', 'norms']
methods = ['OneClassSVM', 'PCA', 'norms', 'quadrics', 'quadrics_alg']
datasets = ['calfw', 'flickr', 'ms1m', 'cplfw', 'megaface']

In [3]:
! python3 create_models.py --methods {' '.join(methods_train)}
! python3 calculate_features.py --shuffle --methods {' '.join(methods)} --datasets {' '.join(datasets)} 

outliers: 100%|###################################| 6/6 [05:41<00:00, 56.84s/it]


### Experiments

We measure distance between the distributions using the ROC-AUC score (for the binary classifier that discriminates these two distributions).

For each dataset we perform `config_test['n_experiments']` experiments, getting the mean and the 0.95-confidence interval.

In [8]:
def get_rocauc(distances_id, distances_ood):
    """Return roc score with 2 arrays as in-distribution
       and out-of-distribution arrays respectively"""
    y = np.array([1]*len(distances_id) + [0]*len(distances_ood))
    return roc_auc_score(1-y, np.concatenate((distances_id, distances_ood), axis=0))


def roc_auc_experiments(in_distr_features, ood_features, in_distr_fraction=config_test['proportion_of_outliers'],
                       n_experiments=config_test['n_experiments'], shuffle=False, extra_params=None):
    
    result = []
    emb_length = int(len(ood_features) / in_distr_fraction)
    
    for i in range(n_experiments):
        if shuffle:
            test_emb = in_distr_features[np.random.choice(len(in_distr_features), size=emb_length)]
        else:
            assert emb_length * n_experiments == len(in_distr_features)
            test_emb = in_distr_features[i*emb_length:(i+1)*emb_length]
            
        result.append(get_rocauc(test_emb, ood_features))
        
    return np.mean(result), np.std(result)

In [9]:
results = {}

for method in methods:
    ood_features = np.load('features/outliers/{}_dist.npy'.format(method))
    results[method] = {}
    for dataset in datasets:
        in_distr_features = np.load('features/{}/{}_dist.npy'.format(dataset, method))
        mean, std = roc_auc_experiments(in_distr_features, ood_features)
        conf_intv = 1.96 * std / np.sqrt(config_test['n_experiments'])
        
        results[method][dataset] = (np.round(mean, 3), np.round(conf_intv, 3))

In [78]:
# rename dict
result_renamed = {columns_dict[method]: {datasets_rename[dataset_name]: results[method][dataset_name] 
                        for dataset_name in results[method]} for method in results}

results_df = pd.DataFrame.from_dict(result_renamed)

### Results


In [98]:
results_df.sort_index(key= lambda x: [dataset_order[i] for i in x])

,OCSVM,PCA,NORM,Q-FULL,Q-SUB
MS1M-ArcFace,"(0.776, 0.0)","(0.883, 0.0)","(0.752, 0.0)","(0.972, 0.0)","(0.82, 0.0)"
MegaFace,"(0.698, 0.001)","(0.808, 0.0)","(0.742, 0.0)","(0.885, 0.0)","(0.763, 0.001)"
FFQH,"(0.707, 0.0)","(0.898, 0.0)","(0.72, 0.0)","(0.924, 0.0)","(0.823, 0.0)"
CPLFW,"(0.774, 0.0)","(0.819, 0.0)","(0.747, 0.0)","(0.922, 0.0)","(0.727, 0.0)"
CALFW,"(0.788, 0.0)","(0.878, 0.0)","(0.703, 0.0)","(0.973, 0.0)","(0.789, 0.0)"


#### Remark
The results for OCSVM in the table above do not need to coincide with the corresponding results from Table 1 in the paper. In this notebook we used a data sample of size 2000 to train OCSVM while for Table 1 we used a data sample of size 80000 to train OCSVM. Also, results for some other methods may differ from Table 1 by a tiny bit. This is caused by the different number of experiments they were averaged over (20 here vs 10 in the paper).